<a href="https://colab.research.google.com/github/ConradBitt/processamento_linguagem_natural/blob/master/Word2vec_Treinando_word_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução 

Este notebook tem como objetivo criar representações textuais através de **word2vec**, um classificador de títulos de reportagens de jornal. Posteriormente, após a representação, treino e teste do modelo, será disponibilizado através de uma aplicação web visando simular um ambiente de produção.

## Montando Drive

O motivo de usar o colaboratory é porque processamento de linguagem envolve muita memoria RAM e portanto rodar localmente pode se tonar um tanto custoso.Além disso, através do calab é permitido montar uma imagem do drive onde podemos armazenar os dados. Por esses dois motivos optei pelo google colab:


In [1]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd 
import seaborn as sns 
import numpy as np
import matplotlib as mpl
import spacy


from matplotlib import pyplot as plt

sns.set_context('talk')

# Executar apenas uma vez e depois reiniciar o ambiente de execução.
#!python -m spacy download pt_core_news_sm

print('~ Versões dos Módulos ~')
print(f'Pandas: {pd.__version__}')
print(f'Seaborn: {sns.__version__}')
print(f'Numpy: {np.__version__}')
print(f'Matplotlib: {mpl.__version__}')
print(f'Spacy: {spacy.__version__}')

~ Versões dos Módulos ~
Pandas: 1.1.5
Seaborn: 0.11.1
Numpy: 1.19.5
Matplotlib: 3.2.2
Spacy: 2.2.4


# Lendo os dados

In [3]:
treino = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/word2vec/dados/treino.csv')
teste = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/word2vec/dados/teste.csv')

## Informações datasets

In [4]:
display(treino.info())
display(teste.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90000 entries, 0 to 89999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        90000 non-null  object
 1   text         90000 non-null  object
 2   date         90000 non-null  object
 3   category     90000 non-null  object
 4   subcategory  17175 non-null  object
 5   link         90000 non-null  object
dtypes: object(6)
memory usage: 4.1+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20513 entries, 0 to 20512
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        20513 non-null  object
 1   text         20513 non-null  object
 2   date         20513 non-null  object
 3   category     20513 non-null  object
 4   subcategory  6794 non-null   object
 5   link         20513 non-null  object
dtypes: object(6)
memory usage: 961.7+ KB


None

Temos: 
* 90000 observações nos dados de treino, totalizando 4.1MB.
* 20513 observações nos dados de teste, totalizando 961KB

todos os dados são do tipo object, porvavelmente strings dividos em 6 variaveis, `titulo`, `text`, `date`, `category`, `subcategory`, `link`.

## Exibindo alguns dados

In [5]:
treino.head()

,title,text,date,category,subcategory,link
0,"Após polêmica, Marine Le Pen diz que abomina n...",A candidata da direita nacionalista à Presidên...,2017-04-28,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
1,"Macron e Le Pen vão ao 2º turno na França, em ...",O centrista independente Emmanuel Macron e a d...,2017-04-23,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
2,"Apesar de larga vitória nas legislativas, Macr...",As eleições legislativas deste domingo (19) na...,2017-06-19,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/06/189...
3,"Governo antecipa balanço, e Alckmin anuncia qu...",O número de ocorrências de homicídios dolosos ...,2015-07-24,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/07...
4,"Após queda em maio, a atividade econômica sobe...","A economia cresceu 0,25% no segundo trimestre,...",2017-08-17,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/08/1...


Usaremos a variável ``title`` para tentar estimar a variável ``category``. Entretanto, vamos analisar como é apresentada uma observação:


In [6]:
treino.title.iloc[-10]

'Pagamento de propina pode chegar a R$ 50 mil por carga'

perceba que temos palavras, números e caracteres especiais "\$", então é necessário um pré processamento.

# Pre processamento 

Geralmente se utiliza o NLTK para fazer a filtragem de caracteres e tokenização. Neste notebook vou utilizar o [spaCy](https://spacy.io/) um módulo escrito em  Python e Cython para processamento de linguagem natural.

A documentação esta disponível em: 

> https://spacy.io/api



## Instalando SpaCy

Para instalar o SpaCy segue-se as instruções no site:

> https://spacy.io/usage

Entretanto no colab ele já vem instalado por padrão

O que precisamo fazer é baixar a lingua cujo os nossos dados foram escritos, no mesmo site tem as linhas que devem ser executadas para instalar o idioma portguês:

In [7]:
# Executar apenas uma vez e depois reiniciar o ambiente de execução.
#!python -m spacy download pt_core_news_sm

Para testar se o idioma desejado foi instalado executamos o comando. **Depois de baixar os dados é necessário reiniciar o ambiente de execução.**

In [8]:
nlp = spacy.load('pt_core_news_sm')

### Exemplo de uso

In [9]:
meu_texto = treino.title.iloc[22]
print(meu_texto)

doc = nlp(meu_texto)
print(type(doc))

Jon Snow volta aos holofotes em vídeo da 6ª temporada de 'Game of Thrones'
<class 'spacy.tokens.doc.Doc'>


Note que o nlp tem objetos do tipo ``Doc``, para saber o que esse objeto contém, podemos analisar a documentação:

> https://spacy.io/usage/spacy-101#pipelines

A descrição do Doc é: 

> A Doc is a sequence of Token objects. Access sentences and named entities, export annotations to numpy arrays, losslessly serialize to compressed binary strings. The Doc object holds an array of TokenC structs. The Python-level Token and Span objects are views of this array, i.e. they don’t own the data themselves. Fonte: [https://spacy.io/api/doc](https://spacy.io/api/doc)

Basicamente quando instanciamos um objeto ``Doc`` o ``nlp`` faz uma série de processamentos, como tokenização, targge, parser, entre outros, até criar o objeto.

<img src='https://spacy.io/pipeline-fde48da9b43661abcdf62ab70a546d71.svg' width=80%>

> Fonte: https://spacy.io/usage/spacy-101#pipelines

Os objetos ``Doc`` tem atributos muito uteis, podemos consultar em

> https://spacy.io/api/doc


In [10]:
print('Meu Doc: ', doc)
print('Entidades de doc: ', doc.ents)


Meu Doc:  Jon Snow volta aos holofotes em vídeo da 6ª temporada de 'Game of Thrones'
Entidades de doc:  (Jon Snow, Game of Thrones)


O objeto consegue identificar varias classificações dos elementos que compõem a frase

In [11]:
print(doc.text, '\n')
for token in doc:
    print(token.text, token.pos_, token.dep_)

Jon Snow volta aos holofotes em vídeo da 6ª temporada de 'Game of Thrones' 

Jon PROPN nsubj
Snow PROPN flat:name
volta VERB ROOT
a ADP case
os DET det
holofotes PROPN obl
em ADP case
vídeo NOUN obl
da ADP case
6ª ADJ amod
temporada NOUN nmod
de ADP case
' PUNCT punct
Game PROPN nmod
of PROPN flat:name
Thrones PROPN flat:name
' PUNCT punct


é possível utilizar varios métodos dos tokens, por exemplo a representação da palavra em um espaço vetorial:

In [12]:
print(doc[1], '=', doc[1].vector)

print('\nNorma do vetor')
print(doc[1], '=', doc[1].vector_norm)

Snow = [  2.7633104    7.462899     0.63145417   0.7640705    1.4044249
   1.0847819   -5.712425    -2.5970128    3.591257     6.957478
  11.282299     1.5133985   -5.667033    -1.6723096   -0.7537353
  -1.4369892   -2.0909038    0.29047918   4.8055325   -4.069366
  -3.749087    -1.4293463   -2.5723786    0.60486764  -4.8334146
  -7.603295     0.38302672  -3.394683    -1.864464    -7.2505016
  -2.334701    -5.8551       7.2367682    3.8455513    1.262495
 -10.394055     3.4092844   -4.875017     0.09814078   5.481478
   1.432259    -1.1912625   -7.294257    -2.0385437   -1.3923185
  -0.9888474   -4.113654     2.6139627   -1.9440972    3.370472
  -3.3382788   -8.765093     0.6188494    2.0189853   -7.73819
   9.392529     0.10228646   2.750429     1.814964    -2.7373686
  -1.5277661   -2.0538282    9.940031    -1.2688947    6.332727
  -8.138163     2.3974452   -0.10324526  -2.9152355   -0.81570804
   2.6807451   -5.3069367    4.2606955   -7.808796    -0.45525482
   2.3871043    1.440799

Podemos fazer análise de sentimento em relação ao ``token``:

> ``sentiment``	A scalar value indicating the positivity or negativity of the token. ``float``

In [13]:
for token in doc:
  print(token.sentiment)


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


existem varios outros atributos e métodos disponíveis para tokens, disponíveis na documentação:

> https://spacy.io/api/token
  

## Algumas modificações e Generator Expression

Precisamos dos títulos tratados adequadamente para ser utilizados em modelos. Para isso é necessario uma função que faça essa modificação.

Antes disso vou deixar todos os dados em minusculo pois não faz diferença entre "comissão" e "Comissão".

> **Em vez de usar list comprehensions utilizarei um gerador criado através do generator expression. Basicamente cria um gerador para ser utilizado em tempo de execução. Para saber mais: [List Comprehension vs Generator Expressions in Python](https://code-maven.com/list-comprehension-vs-generator-expression)**

In [24]:

texto_para_tratamento = (titulo.lower() for titulo in treino.title)
print(f'O gerador: {texto_para_tratamento}')

O gerador: <generator object <genexpr> at 0x7fd08db985d0>


em vez de receber um gerador a função para tratar os textos vai receber um ``Doc`` para conseguirmos realizar as seguintas rotinas:

* Remover stopwords "O, A, do, da, na, se, aqueles", palavras irrelevantes para entender o contexto.

* Ter apenas caracteres alphabeticos 

* Título com no mínomo três palavras.

In [31]:
def trata_textos(doc):
  """
  Esta função trata os textos recebidos,
  remove stopwords, retorna alphanumericos
  maior que três palavras.
  """
  tokens_validos = []
  for token in doc: 
    # variável de validação
    # garante que o token não é stopword e é alphanumerico
    e_valido = not token.is_stop and token.is_alpha
    if e_valido:
      tokens_validos.append(token.text)
      
  if len(tokens_validos) > 2:
    return " ".join(tokens_validos)

Testando a função:

In [30]:
print(doc)
trata_textos(doc)

Jon Snow volta aos holofotes em vídeo da 6ª temporada de 'Game of Thrones'


'Jon Snow volta a holofotes vídeo temporada Game of Thrones'

Um ponto importante é que o ``doc`` que criamos recebeu uma ``string``, mas precisamos passar uma estrtura de dados para ``nlp``, para saber como fazer isso precisamos entender como funciona a classe ``nlp``:

In [32]:
type(nlp)

spacy.lang.pt.Portuguese

Olhando a [documentação do ``spacy.lang``](https://spacy.io/api/language#init) um dos parâmetros utilizados pelo ``__init__()`` é o ``batch_size`` e para passar uma estrutura de dados com textos para o ``nlp`` utiliza-se um "pipeline" uma linha de processos, o seguinte exemplo ilustra como fazer isso: 

```python
texts = ["One document.", "...", "Lots of documents"]
for doc in nlp.pipe(texts, batch_size=50):
    assert doc.has_annotation("DEP")
```

Além do ``batch_size`` passado ao  ``nlp.pipe``, podemos dizer quantos batchs serão processados por cada núcleo através do parâmetro ``n_processor`` descrito na documentação do pipe:

> https://spacy.io/api/language#pipe

> ``n_process`` 	Number of processors to use. Defaults to 1.
int

para ver quantos núcleos de processamento a máquina virtual do colab basta usar o comando
> ``lscpu | grep -E '^Thread|^Core|^Socket|^CPU\('``

In [38]:
!lscpu | grep -E '^Thread|^Core|^Socket|^CPU\('

CPU(s):              2
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1


caso não saiba quantos núcleos de processamento tem, é possível passar ``n_process=-1`` ai o spacy irá utilizar todos os disponíveis.

<font color=red ><b>Lembrando que esse processo pode demorar!</b></font>

In [40]:
texto_tratado = [trata_textos(doc) for doc in nlp.pipe(texto_para_tratamento,
                                                      batch_size=1000,
                                                      n_process = -1)]

CPU times: user 1h 54min 5s, sys: 1min 22s, total: 1h 55min 28s
Wall time: 2h 1min 1s



### Salvando objeto
vou salvar est objeto para evitar que demore todo este tempo toda vez que for necessario executar este notebook:


In [57]:
import pickle

file_texto_tratado = open(b'texto_tratado.obj','wb')
pickle.dump(texto_tratado, file_texto_tratado)

### Lendo objeto salvo

para ler o objeto salvo através do pickle, usamos a seguinte sintaxe, depois de baixar o arquivo:

https://github.com/ConradBitt/processamento_linguagem_natural/blob/master/texto_tratado.obj


In [70]:
import pickle

texto_tratado_objeto = open(b'texto_tratado.obj', 'rb')
texto_tratado_lido_pickle = pickle.load(texto_tratado_objeto)

In [71]:
texto_tratado_lido_pickle[:15]

['ataques multiplicam israel e cisjordânia morrem gaza',
 'dutra trânsito lento chegada a paulo',
 'alta chicago bulls felício destaque brasil nba',
 'sábato influenciou críticos hoje questionado',
 'diretor israelense humor superar a dor perda shivá',
 'continuação universidade columbia analisa erros rolling stone reportagem',
 'el niño trará impactos enormes alertam cientistas',
 'folha lança aplicativo realidade virtual filme percorre sp',
 'platini suspensão fraude e promete recorrer uefa francês',
 'premiê israel pressionado a cancelar visita donald trump',
 None,
 'grêmio chega a acordo barrios e espera liberação palmeiras',
 'perspectivas abrem o mercosul',
 'governo campanha prevenção a hiv aplicativo gay',
 'brilho eterno mente lembranças terá série tv site']

> **Referência**: https://www.greelane.com/pt/ci%C3%AAncia-tecnologia-matem%C3%A1tica/ci%C3%AAncia-da-computa%C3%A7%C3%A3o/using-pickle-to-save-objects-2813661

## Tratando entradas 

In [72]:
titulos_tratados = pd.DataFrame(
    {'titulo':texto_tratado}
)

titulos_tratados.head()

,titulo
0,ataques multiplicam israel e cisjordânia morre...
1,dutra trânsito lento chegada a paulo
2,alta chicago bulls felício destaque brasil nba
3,sábato influenciou críticos hoje questionado
4,diretor israelense humor superar a dor perda s...


In [73]:
titulos_tratados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86000 entries, 0 to 85999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   titulo  82479 non-null  object
dtypes: object(1)
memory usage: 672.0+ KB


# Treinando modelo Gensim